In [1]:
from exp8_feature_extraction import get_balanced_dataset
from scripts.cross_validate import run_cross_validate

import numpy as np
import gensim

import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
all_reviews = get_balanced_dataset()

In [3]:
len(all_reviews)

160933

In [4]:
reviews_contents = [x.review_content for x in all_reviews]
labels = [1 if x.label else 0 for x in all_reviews]

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(reviews_contents)
input_features = np.array(pad_sequences(tokenizer.texts_to_sequences(reviews_contents)))

In [ ]:
corpus_words = tokenizer.word_index
corpus_vocab_size = len(corpus_words)+1

In [ ]:
word_vectors = gensim.models.KeyedVectors.load_word2vec_format("../../data/GoogleNews-vectors-negative300.bin",
                                                               binary=True)

In [ ]:
embedding_length = word_vectors.vector_size
embedding_matrix = np.zeros((corpus_vocab_size, embedding_length))
for word, index in corpus_words.items():
  if word in word_vectors.vocab:
    embedding_matrix[index] = np.array(word_vectors[word], dtype=np.float32)

In [ ]:
max_review_len = 0
for input_feature in input_features:
    max_review_len = max(max_review_len, len(input_feature))
print(max_review_len)

997


In [ ]:
corpus_vocab_size

103804

In [ ]:
len([x.label for x in all_reviews if x.label == 0])

80467